In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(device)

cuda


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1,28,28,device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(y_pred)

tensor([6], device='cuda:0')


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(hidden1)

tensor([[ 0.3507,  0.1637,  0.1736,  0.3872,  0.2286,  0.4055, -0.0542,  0.3360,
          0.0792,  0.0527, -0.1724,  0.2778, -0.0231, -0.2689,  0.6064,  0.1132,
         -0.0128,  0.3137, -0.3627,  0.1695],
        [ 0.3522,  0.3865,  0.3181,  0.3676,  0.2636,  0.2323, -0.1791,  0.2600,
          0.0255,  0.1345, -0.2675, -0.1715,  0.3610, -0.3060,  0.4354, -0.1203,
          0.2554,  0.3181, -0.1662,  0.0795],
        [ 0.3245,  0.6241,  0.5355,  0.3640,  0.5451,  0.2928, -0.0530, -0.1418,
         -0.0625, -0.0619, -0.5029,  0.3923,  0.0948, -0.1066,  0.1772,  0.0353,
         -0.1136,  0.1982, -0.0735,  0.1202]], grad_fn=<AddmmBackward0>)


In [11]:
hidden1 = nn.ReLU()(hidden1)
print(hidden1)

tensor([[0.3507, 0.1637, 0.1736, 0.3872, 0.2286, 0.4055, 0.0000, 0.3360, 0.0792,
         0.0527, 0.0000, 0.2778, 0.0000, 0.0000, 0.6064, 0.1132, 0.0000, 0.3137,
         0.0000, 0.1695],
        [0.3522, 0.3865, 0.3181, 0.3676, 0.2636, 0.2323, 0.0000, 0.2600, 0.0255,
         0.1345, 0.0000, 0.0000, 0.3610, 0.0000, 0.4354, 0.0000, 0.2554, 0.3181,
         0.0000, 0.0795],
        [0.3245, 0.6241, 0.5355, 0.3640, 0.5451, 0.2928, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.3923, 0.0948, 0.0000, 0.1772, 0.0353, 0.0000, 0.1982,
         0.0000, 0.1202]], grad_fn=<ReluBackward0>)


In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [15]:
print({model})
for name,param in model.named_parameters():
    print(name,param.size(),param[:2])

{NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)}
linear_relu_stack.0.weight torch.Size([512, 784]) tensor([[ 7.7978e-03, -4.3743e-03, -9.8696e-03,  ...,  9.2220e-03,
         -1.1964e-05,  1.9522e-02],
        [ 2.4185e-02,  6.4444e-04,  2.7597e-02,  ...,  1.1924e-03,
         -3.0093e-03, -3.0487e-02]], device='cuda:0', grad_fn=<SliceBackward0>)
linear_relu_stack.0.bias torch.Size([512]) tensor([-0.0022, -0.0041], device='cuda:0', grad_fn=<SliceBackward0>)
linear_relu_stack.2.weight torch.Size([512, 512]) tensor([[ 0.0383,  0.0408, -0.0073,  ...,  0.0078,  0.0428,  0.0363],
        [ 0.0146, -0.0166,  0.0131,  ..., -0.0082,  0.0381,  0.0275]],
       device='cuda:0', grad_fn=<SliceBackward0>)
linear_relu_stack.2.bias 